In [1]:
%matplotlib qt

import hyperspy.api as hs
import pyxem as pxm
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib as mpl

from pathlib import Path

from matplotlib.colors import SymLogNorm, to_rgba, LinearSegmentedColormap

from skimage.filters.thresholding import try_all_threshold, threshold_triangle, threshold_li, threshold_isodata
from skimage.measure import label

color_names = ['linen', 'darkorange', 'dodgerblue', 'forestgreen', 'red']
colors = [to_rgba(c) for c in color_names]

cmap = LinearSegmentedColormap.from_list('gt_cmap', colors, N=len(color_names))

gray_cmap = plt.colormaps.get('Greys')
gray_cmap.set_bad('lightblue')

In [29]:
from functools import reduce

In [49]:
def union_mask(masks, invert=True):
    """
    Return the union between masks in a list
    
    Masks should be `True` wherever data is to be be excluded in processing/analysis. 
    
    Arguments:
    ----------
    masks: A list of hyperspy 2DSignal objects of shape (|nx, ny).
    invert: Whether to invert the masks before taking the union (usually needed for hyperspy masks). The final mask will be re-inverted before returning.
    
    Returns:
    --------
    mask: The union of the provided masks
    """
    if invert:
        masks = [~mask for mask in masks]
    
    mask = masks.pop()
    for m in masks:
        mask = mask & m
    
    if invert:
        mask = ~mask
        
    mask.metadata.General.title = 'Mask'
        
    return mask
    

# Dataset A

## Load and prepare data

In [3]:
filepath = Path(r'Dataset A/datasetA_preprocessed.hspy')

In [4]:
s = hs.load(str(filepath), lazy=False)
s.change_dtype('float32')
vbf = s.get_integrated_intensity(hs.roi.CircleROI(0.0, 0.0, 0.1))
maximums = s.max(axis=[0, 1])
try:
    vbf.compute()
    maximums.compute()
except AttributeError:
    pass
hs.plot.plot_images([vbf, maximums], norm='symlog', axes_decor='off', colorbar=None, cmap='gray_r')

[<AxesSubplot: title={'center': 'Integrated intensity'}, xlabel='x axis (nm)', ylabel='y axis (nm)'>,
 <AxesSubplot: title={'center': 'Dataset A'}, xlabel='kx axis ($A^{-1}$)', ylabel='ky axis ($A^{-1}$)'>]

In [5]:
s.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 23.036135801743175
│       │       └── exposure_time = 10
│       ├── beam_energy = 200
│       ├── rocking_angle = 1.04
│       ├── rocking_frequency = 100
│       └── scan_rotation = 0
├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 1.7.3
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-03-22T08:20:25.199003+01:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 1.7.3
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2023-03-22T08:32:52.490161+01:00
│   │   ├── 2
│   │   │   ├── hyperspy_version = 1.7.3
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-03-22T08:45:26.118812+01:00
│   │   └── 3
│   │       ├── hyperspy_version = 1.7.3
│   │       ├── io_plugin = hyperspy.io_plugins.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2023-04-11T10:05:44.416236+02:00
│   └── title = Dataset A
├── Preprocessing
│   ├── Centering
│   │   ├── COM = <DPCSignal2D, title: , dimensions: (2|512, 512)>
│   │   ├── COM_mask
│   │   │   ├── r = 12.5
│   │   │   ├── x = 127
│   │   │   └── y = 126
│   │   ├── Shifts = <BeamShift, title: , dimensions: (512, 512|2)>
│   │   └── shift_estimate_mask = <BaseSignal, title: , dimensions: (512, 512|)>
│   └── Masks
│       └── Diffraction
│           ├── cutoff = <Signal2D, title: <7.001e-01 $A^{-1}$ mask, dimensions: (|128, 128)>
│           ├── direct_beam = <Signal2D, title: >1.595e-01 $A^{-1}$ mask, dimensions: (|128, 128)>
│           └── reflections = <Signal2D, title: Reflection mask, dimensions: (|128, 128)>
└── Signal
    └── signal_type = electron_diffraction

## Get the pre-made masks from the metadata (see preprocessing notebook for details)

In [51]:
diffmask = union_mask([mask for (_, mask) in s.metadata.Preprocessing.Masks.Diffraction])
hs.plot.plot_images([maximums, diffmask*1.0], overlay=True, alphas=[1, 0.5], colors=['w', 'r'], axes_decor='off')

[<Axes: >]

# First iteration
## Run SVD decomposition

In [52]:
tic = time.time()   
decomp = s.decomposition(
    normalize_poissonian_noise=True,
    algorithm='SVD',
    signal_mask=diffmask.data,
    return_info=True,
    copy=True
)
toc = time.time()
print(f'Finished decomposition. Elapsed time: {toc - tic} seconds')
s.learning_results.save(filepath.with_name(f'{filepath.stem}_SVD1'))

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=None
  centre=None
Finished decomposition. Elapsed time: 1443.7520871162415 seconds


In [53]:
threshold = 5
#where the estimated threshold in the explained variance is - tune!
SMALL_SIZE = 6
MEDIUM_SIZE = 8
LARGE_SIZE = 10
with mpl.rc_context({
    'font.size': SMALL_SIZE, 
    'axes.titlesize': SMALL_SIZE,
    'axes.labelsize': MEDIUM_SIZE,
    'xtick.labelsize': SMALL_SIZE,
    'ytick.labelsize': SMALL_SIZE,
    'legend.fontsize': SMALL_SIZE,
    'figure.titlesize': LARGE_SIZE
}):
    markersize = 4 #Marker size for the plot
    dpi=300 #DPI
    figwidth = 468/3 #figure size in points
    pt2in = 0.01389 #conversion from points to inches
    figsize = (figwidth*pt2in, figwidth*pt2in) #Figuresize in inches
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    s.plot_explained_variance_ratio(n=32,
                                    threshold=threshold,
                                    xaxis_type='number',
                                    xaxis_labeling='ordinal',
                                    signal_fmt={'marker': 'o', 'color': 'k', 'markerfacecolor': 'k', 'lw': 0, 'ms':markersize},
                                    noise_fmt={'marker':'o', 'color': 'k', 'markerfacecolor': 'w', 'lw': 0, 'ms':markersize},
                                    fig=fig,
                                    ax=ax
                                   )
    ax.set_title('')
    ax.set_xlim(0)
    plt.tight_layout()
    fig.savefig(filepath.with_name(f'{filepath.stem}_SVD1.png'), dpi=dpi)

In [54]:
phases = {
    'theta_100': {'value': 1.0, 'components': tuple(), 'thresholds': {}},
    'theta_001': {'value': 2.0, 'components': tuple(), 'thresholds': {}},
    'T1': {'value': 3.0, 'components': tuple(), 'thresholds': {}}
}


## Run first NMF decomposition

In [55]:
s = hs.load(str(filepath), lazy=False) #Reload data without any decomposition results
s.change_dtype('float32')

output_dimension = 5 #The number of components to allow
tic = time.time()
decomp = s.decomposition(
    normalize_poissonian_noise=True,
    algorithm='NMF',
    output_dimension=output_dimension,
    signal_mask=diffmask,
    return_info=True,
    init='nndsvd',
    max_iter=10000
)
toc = time.time()
print(f'Finished decomposition. Elapsed time: {toc - tic} seconds')
print(f'Decoposition parameters: {decomp}')
print(f'Decomposition reconstruction error: {decomp.reconstruction_err_}')
print(f'Decomposition number of iterations: {decomp.n_iter_}')

#Save the decomposition results
s.learning_results.save(filepath.with_name(f'{filepath.stem}_NMF1'), overwrite=True)

#Save the factors and loadings individually as well
factors = s.get_decomposition_factors()
loadings = s.get_decomposition_loadings()
if decomp is not None:
    factors.metadata.add_dictionary({'Decomposition': decomp.__dict__})
    loadings.metadata.add_dictionary({'Decomposition': decomp.__dict__})
factors.save(filepath.with_name(f'{filepath.stem}_NMF1_{output_dimension}_factors.hspy'), overwrite=True)
loadings.save(filepath.with_name(f'{filepath.stem}_NMF1_{output_dimension}_loadings.hspy'), overwrite=True)

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=5
  centre=None
scikit-learn estimator:
NMF(init='nndsvd', max_iter=10000, n_components=5)
Finished decomposition. Elapsed time: 5572.3297798633575 seconds
Decoposition parameters: NMF(init='nndsvd', max_iter=10000, n_components=5)
Decomposition reconstruction error: 0.16367157842750354
Decomposition number of iterations: 8401


In [56]:
hs.plot.plot_images(loadings, per_row=output_dimension, cmap='gray_r', axes_decor='off', colorbar=None)
hs.plot.plot_images(factors, per_row=output_dimension, cmap='gray_r', norm='symlog', axes_decor='off', colorbar=None)

[<AxesSubplot: title={'center': ' (0,)'}, xlabel='kx axis ($A^{-1}$)', ylabel='ky axis ($A^{-1}$)'>,
 <AxesSubplot: title={'center': ' (1,)'}, xlabel='kx axis ($A^{-1}$)', ylabel='ky axis ($A^{-1}$)'>,
 <AxesSubplot: title={'center': ' (2,)'}, xlabel='kx axis ($A^{-1}$)', ylabel='ky axis ($A^{-1}$)'>,
 <AxesSubplot: title={'center': ' (3,)'}, xlabel='kx axis ($A^{-1}$)', ylabel='ky axis ($A^{-1}$)'>,
 <AxesSubplot: title={'center': ' (4,)'}, xlabel='kx axis ($A^{-1}$)', ylabel='ky axis ($A^{-1}$)'>]

### Estimate thresholds for phase map

In [57]:
phases['T1']['components'] = (1, 3)
phases['theta_100']['components'] = (2, 4)

In [71]:
estimate_threshold(loadings, 3)

skimage.filters.thresholding.threshold_isodata
skimage.filters.thresholding.threshold_li
skimage.filters.thresholding.threshold_mean
skimage.filters.thresholding.threshold_minimum
skimage.filters.thresholding.threshold_otsu
skimage.filters.thresholding.threshold_triangle
skimage.filters.thresholding.threshold_yen


In [78]:
phases['T1']['thresholds'].update({component: estimate_threshold(loadings, component, threshold_triangle) for component in phases['T1']['components']})
phases['theta_100']['thresholds'].update({component: estimate_threshold(loadings, component, threshold_li) for component in phases['theta_100']['components']})

In [79]:
phases

{'theta_100': {'value': 1.0,
  'components': (2, 4),
  'thresholds': {2: 0.00812242, 4: 0.007290254}},
 'theta_001': {'value': 2.0, 'components': (), 'thresholds': {}},
 'T1': {'value': 3.0,
  'components': (1, 3),
  'thresholds': {1: 0.0017662686, 3: 0.0020232506}}}

### Apply thresholds and make phase maps

In [102]:
#Create mask signals
theta_100_mask = hs.signals.Signal2D(np.zeros(s.axes_manager.navigation_shape, dtype=bool))
T1_mask = hs.signals.Signal2D(np.zeros(s.axes_manager.navigation_shape, dtype=bool))
for ax_no, ax in enumerate(s.axes_manager.navigation_axes):
    for mask in [T1_mask, theta_100_mask]:
        mask.axes_manager[ax_no].name = s.axes_manager[ax].name
        mask.axes_manager[ax_no].scale = s.axes_manager[ax].scale
        mask.axes_manager[ax_no].units = s.axes_manager[ax].units
        mask.axes_manager[ax_no].offset = s.axes_manager[ax].offset
        
#Set mask data for theta_100 
for component in phases['theta_100']['components']:
    theta_100_mask += loadings.inav[component]>=phases['theta_100']['thresholds'][component]
theta_100_mask.metadata.General.title = 'theta_100'
theta_100 = theta_100_mask * phases['theta_100']['value']
theta_100.plot()

#Set mask data for T1
for component in phases['T1']['components']:
    T1_mask += loadings.inav[component]>=phases['T1']['thresholds'][component]
T1_mask.metadata.General.title = 'T1'
T1 = T1_mask * phases['T1']['value']
T1.plot()

hs.plot.plot_images([T1, theta_100], overlay=True, colors=[colors[-2], colors[-4]])

if True:
    theta_100.save(filepath.with_name(f'{filepath.stem}_theta_100.hspy'))
    T1.save(filepath.with_name(f'{filepath.stem}_T1.hspy'))

    [s.metadata.add_dictionary({'Preprocessing': {'Masks': {'Navigation': {mask.metadata.General.title: mask}}}}) for mask in [theta_100_mask, T1_mask]]

    hs.plot.plot_images([vbf, T1, theta_100], overlay=True, alphas=[1, 1, 1], colors=['w', colors[3], colors[1]], axes_decor='off')
    fig = plt.gcf()
    fig.savefig(filepath.with_name(f'{filepath.stem}_T1_theta100.png'))

Overwrite '/mnt/DATA1/Emil/PhaseMappingPaper/Dataset A/datasetA_preprocessed_theta_100.hspy' (y/n)?
 y
Overwrite '/mnt/DATA1/Emil/PhaseMappingPaper/Dataset A/datasetA_preprocessed_T1.hspy' (y/n)?
 y


# 2nd iteration

### get the new masks

In [100]:
navmask = union_mask([mask for (_, mask) in s.metadata.Preprocessing.Masks.Navigation])
hs.plot.plot_images([vbf, navmask*1.0], overlay=True, alphas=[1, 0.5], colors=['w', 'r'], axes_decor='off')
fig = plt.gcf()
fig.savefig(filepath.with_name(f'{filepath.stem}_NMF2_navmask.png'))

## SVD

In [154]:
s = hs.load(str(filepath), lazy=False) #Reload data without any decomposition results
s.change_dtype('float32')

tic = time.time()
decomp = s.decomposition(
    normalize_poissonian_noise=True,
    algorithm='SVD',
    signal_mask=diffmask.data,
    navigation_mask=navmask.data.T,
    return_info=True
)
toc = time.time()
print(f'Finished decomposition. Elapsed time: {toc - tic} seconds')
s.learning_results.save(filepath.with_name(f'{filepath.stem}_SVD2'), overwrite=True)

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=None
  centre=None
Finished decomposition. Elapsed time: 1264.411277770996 seconds


In [155]:
threshold = 6
#where the estimated threshold in the explained variance is - tune!
SMALL_SIZE = 6
MEDIUM_SIZE = 8
LARGE_SIZE = 10
with mpl.rc_context({
    'font.size': SMALL_SIZE, 
    'axes.titlesize': SMALL_SIZE,
    'axes.labelsize': MEDIUM_SIZE,
    'xtick.labelsize': SMALL_SIZE,
    'ytick.labelsize': SMALL_SIZE,
    'legend.fontsize': SMALL_SIZE,
    'figure.titlesize': LARGE_SIZE
}):
    markersize = 4 #Marker size for the plot
    dpi=300 #DPI
    figwidth = 468/3 #figure size in points
    pt2in = 0.01389 #conversion from points to inches
    figsize = (figwidth*pt2in, figwidth*pt2in) #Figuresize in inches
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    s.plot_explained_variance_ratio(n=32,
                                    threshold=threshold,
                                    xaxis_type='number',
                                    xaxis_labeling='ordinal',
                                    signal_fmt={'marker': 'o', 'color': 'k', 'markerfacecolor': 'k', 'lw': 0, 'ms':markersize},
                                    noise_fmt={'marker':'o', 'color': 'k', 'markerfacecolor': 'w', 'lw': 0, 'ms':markersize},
                                    fig=fig,
                                    ax=ax
                                   )
    ax.set_title('')
    ax.set_xlim(0)
    plt.tight_layout()
    fig.savefig(filepath.with_name(f'{filepath.stem}_SVD2.png'), dpi=dpi)

In [156]:
s.plot_decomposition_results()

### NMF

In [106]:
s = hs.load(str(filepath), lazy=False) #Reload data without any decomposition results
s.change_dtype('float32')

output_dimension = 6 #The number of components to allow
tic = time.time()
decomp = s.decomposition(
    normalize_poissonian_noise=True,
    algorithm='NMF',
    output_dimension=output_dimension,
    signal_mask=diffmask,
    navigation_mask=navmask.data.T,
    return_info=True,
    init='nndsvd',
    max_iter=10000
)
toc = time.time()
print(f'Finished decomposition. Elapsed time: {toc - tic} seconds')
print(f'Decoposition parameters: {decomp}')
print(f'Decomposition reconstruction error: {decomp.reconstruction_err_}')
print(f'Decomposition number of iterations: {decomp.n_iter_}')

#Save the decomposition results
s.learning_results.save(filepath.with_name(f'{filepath.stem}_NMF2'), overwrite=True)

#Save the factors and loadings individually as well
factors = s.get_decomposition_factors()
loadings = s.get_decomposition_loadings()
if decomp is not None:
    factors.metadata.add_dictionary({'Decomposition': decomp.__dict__})
    loadings.metadata.add_dictionary({'Decomposition': decomp.__dict__})
factors.save(filepath.with_name(f'{filepath.stem}_NMF2_{output_dimension}_factors.hspy'), overwrite=True)
loadings.save(filepath.with_name(f'{filepath.stem}_NMF2_{output_dimension}_loadings.hspy'), overwrite=True)

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=6
  centre=None
scikit-learn estimator:
NMF(init='nndsvd', max_iter=10000, n_components=6)
Finished decomposition. Elapsed time: 4677.641726732254 seconds
Decoposition parameters: NMF(init='nndsvd', max_iter=10000, n_components=6)
Decomposition reconstruction error: 0.1301728144520098
Decomposition number of iterations: 9065


In [107]:
hs.plot.plot_signals([loadings, factors])

In [108]:
phases['theta_001']['components'] = (4,)

In [109]:
estimate_threshold(loadings, 4)

skimage.filters.thresholding.threshold_isodata
skimage.filters.thresholding.threshold_li
skimage.filters.thresholding.threshold_mean
skimage.filters.thresholding.threshold_minimum
skimage.filters.thresholding.threshold_otsu
skimage.filters.thresholding.threshold_triangle
skimage.filters.thresholding.threshold_yen


In [110]:
phases['theta_001']['thresholds'].update({component: estimate_threshold(loadings, component, threshold_isodata) for component in phases['theta_001']['components']})

In [111]:
phases

{'theta_100': {'value': 1.0,
  'components': (2, 4),
  'thresholds': {2: 0.00812242, 4: 0.007290254}},
 'theta_001': {'value': 2.0,
  'components': (4,),
  'thresholds': {4: 0.0041912066626537126}},
 'T1': {'value': 3.0,
  'components': (1, 3),
  'thresholds': {1: 0.0017662686, 3: 0.0020232506}}}

In [114]:
#Create mask signals
theta_001_mask = hs.signals.Signal2D(np.zeros(s.axes_manager.navigation_shape, dtype=bool))
for ax_no, ax in enumerate(s.axes_manager.navigation_axes):
    theta_001_mask.axes_manager[ax_no].name = s.axes_manager[ax].name
    theta_001_mask.axes_manager[ax_no].scale = s.axes_manager[ax].scale
    theta_001_mask.axes_manager[ax_no].units = s.axes_manager[ax].units
    theta_001_mask.axes_manager[ax_no].offset = s.axes_manager[ax].offset
        
#Set mask data for theta_100 
for component in phases['theta_001']['components']:
    theta_001_mask += loadings.inav[component]>=phases['theta_001']['thresholds'][component]
theta_001_mask.metadata.General.title = 'theta_001'
theta_001 = theta_001_mask * phases['theta_001']['value']
theta_001.plot()

hs.plot.plot_images([theta_001, T1, theta_100], overlay=True, colors=[colors[-3], colors[-2], colors[-4]])

if True:
    theta_001.save(filepath.with_name(f'{filepath.stem}_theta_001.hspy'))

    s.metadata.add_dictionary({'Preprocessing': {'Masks': {'Navigation': {theta_001_mask.metadata.General.title: theta_001_mask}}}})

    hs.plot.plot_images([vbf, theta_001, T1, theta_100], overlay=True, alphas=[1, 1, 1, 1], colors=['w', colors[-3], colors[-2], colors[-4]], axes_decor='off')
    fig = plt.gcf()
    fig.savefig(filepath.with_name(f'{filepath.stem}_theta001_T1_theta100.png'))

Overwrite '/mnt/DATA1/Emil/PhaseMappingPaper/Dataset A/datasetA_preprocessed_theta_001.hspy' (y/n)?
 y


In [130]:
matrix = hs.signals.Signal2D(np.ones(s.axes_manager.navigation_shape, dtype='uint8'))
for ax_no, ax in enumerate(s.axes_manager.navigation_axes):
    matrix.axes_manager[ax_no].name = s.axes_manager[ax].name
    matrix.axes_manager[ax_no].scale = s.axes_manager[ax].scale
    matrix.axes_manager[ax_no].units = s.axes_manager[ax].units
    matrix.axes_manager[ax_no].offset = s.axes_manager[ax].offset
matrix.metadata.General.title = 'Al'

matrix.data[theta_001_mask] = 0
matrix.data[theta_100_mask] = 0
matrix.data[T1_mask] = 0


vbf.metadata.General.title = 'VBF'

if True:
    matrix.save(filepath.with_name(f'{filepath.stem}_matrix.hspy'))

    s.metadata.add_dictionary({'Preprocessing': {'Masks': {'Navigation': {matrix.metadata.General.title: matrix}}}})

    hs.plot.plot_images([matrix, theta_001, T1, theta_100, vbf], overlay=True, alphas=[1, 1, 1, 1, 1], colors=[colors[0], colors[-3], colors[-2], colors[-4], 'w'], axes_decor='off', legend_loc='lower left')
    fig = plt.gcf()
    fig.savefig(filepath.with_name(f'{filepath.stem}_matrix_theta001_T1_theta100.png'))

Overwrite '/mnt/DATA1/Emil/PhaseMappingPaper/Dataset A/datasetA_preprocessed_matrix.hspy' (y/n)?
 y


In [150]:
phasemap = hs.signals.Signal2D(np.zeros(s.axes_manager.navigation_shape))
phasemap.data[theta_001_mask] = phases['theta_001']['value']
phasemap.data[T1_mask] = phases['T1']['value']
phasemap.data[theta_100_mask] = phases['theta_100']['value']
phasemap.data[matrix] = 1

for ax_no, ax in enumerate(s.axes_manager.navigation_axes):
    phasemap.axes_manager[ax_no].name = s.axes_manager[ax].name
    phasemap.axes_manager[ax_no].scale = s.axes_manager[ax].scale
    phasemap.axes_manager[ax_no].units = s.axes_manager[ax].units
    phasemap.axes_manager[ax_no].offset = s.axes_manager[ax].offset
phasemap.metadata.General.title = 'NMF phasemap'

phasemap.plot(cmap=cmap, vmax=4)

phasemap.save(filepath.with_name(f'{filepath.stem}_phasemap.hspy'))

In [140]:
s.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 23.036135801743175
│       │       └── exposure_time = 10
│       ├── beam_energy = 200
│       ├── rocking_angle = 1.04
│       ├── rocking_frequency = 100
│       └── scan_rotation = 0
├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 1.7.3
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-03-22T08:20:25.199003+01:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 1.7.3
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2023-03-22T08:32:52.490161+01:00
│   │   ├── 2
│   │   │   ├── hyperspy_version = 1.7.3
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-03-22T08:45:26.118812+01:00
│   │   └── 3
│   │       ├── hyperspy_version = 1.7.3
│   │       ├── io_plugin = hyperspy.io_plugins.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2023-04-11T13:59:21.745276+02:00
│   └── title = Dataset A
├── Preprocessing
│   ├── Centering
│   │   ├── COM = <DPCSignal2D, title: , dimensions: (2|512, 512)>
│   │   ├── COM_mask
│   │   │   ├── r = 12.5
│   │   │   ├── x = 127
│   │   │   └── y = 126
│   │   ├── Shifts = <BeamShift, title: , dimensions: (512, 512|2)>
│   │   └── shift_estimate_mask = <BaseSignal, title: , dimensions: (512, 512|)>
│   └── Masks
│       ├── Diffraction
│       │   ├── cutoff = <Signal2D, title: <7.001e-01 $A^{-1}$ mask, dimensions: (|128, 128)>
│       │   ├── direct_beam = <Signal2D, title: >1.595e-01 $A^{-1}$ mask, dimensions: (|128, 128)>
│       │   └── reflections = <Signal2D, title: Reflection mask, dimensions: (|128, 128)>
│       └── Navigation
│           ├── Al = <Signal2D, title: Al, dimensions: (|512, 512)>
│           └── theta_001 = <Signal2D, title: theta_001, dimensions: (|512, 512)>
└── Signal
    └── signal_type = electron_diffraction